In [ ]:
import numpy as np

#HMM

#1. Forward HMM

In [ ]:
def forward(pi, A, B, O):
    T = len(O)
    N = len(pi)
    alpha = np.zeros((T, N))

    # Initialization
    for i in range(N):
        alpha[0, i] = pi[i] * B[i][O[0]]

    # Induction
    for t in range(1, T):
        for j in range(N):
            sum_prev = 0
            for i in range(N):
                sum_prev += alpha[t-1, i] * A[i][j]
            alpha[t, j] = sum_prev * B[j][O[t]]

    likelihood = np.sum(alpha[T-1])
    return alpha, likelihood


#2. Viterbi

In [ ]:
def viterbi(pi, A, B, O):

    T = len(O)     # number of observations
    N = len(pi)    # number of states

    # delta[t][i] = max probability of any path ending in state i at time t
    delta = np.zeros((T, N))

    # psi[t][i] = the state index that gave the best previous step
    psi = np.zeros((T, N), dtype=int)

    # 1. Initialization
    for i in range(N):
        delta[0][i] = pi[i] * B[i][O[0]]
        psi[0][i] = 0

    # 2. Recursion (induction)
    for t in range(1, T):
        for j in range(N):
            best_prev = -1
            best_val = -1

            for i in range(N):
                val = delta[t-1][i] * A[i][j]
                if val > best_val:
                    best_val = val
                    best_prev = i

            delta[t][j] = best_val * B[j][O[t]]
            psi[t][j] = best_prev

    # 3. Tdef expand_with_blanks(labels,blank=0):
  out=[blank]
  for l in labels:
    out.append(l)
    out.append(blank)
  return out

In [ ]:
def main():
    # Example HMM
    pi = [0.7, 0.3]

    A = [
        [0.8, 0.2],
        [0.3, 0.7]
    ]

    B = [
        [0.6, 0.3, 0.1],
        [0.2, 0.5, 0.3]
    ]

    O = [1, 0, 2]  # observation sequence

    delta, psi, best_path, best_prob = viterbi(pi, A, B, O)

    print_table(delta, "Delta (max path probabilities)")
    print_table(psi, "Psi (backpointers)")

    print("Most likely sequence of states:")
    print(best_path)

    print("\nProbability of best path:")
    print(best_prob)


if __name__ == "__main__":
    main()

Delta (max path probabilities):
t = 0: 0.210000 0.150000 
t = 1: 0.100800 0.021000 
t = 2: 0.008064 0.006048 

Psi (backpointers):
t = 0: 0.000000 0.000000 
t = 1: 0.000000 1.000000 
t = 2: 0.000000 0.000000 

Most likely sequence of states:
[np.int64(0), np.int64(0), np.int64(0)]

Probability of best path:
0.008064


---

# CTC

#1. Forward Using log

In [ ]:
import numpy as np
import math

In [ ]:
def logsumexp_vec(vec):
    if len(vec) == 0:
        return -np.inf
    m = np.max(vec)
    if m == -np.inf:
        return -np.inf
    return m + math.log(np.sum(np.exp(vec - m)))


# ----------- expand labels with blanks -------------
def expand_with_blanks(labels, blank=0):
    out = [blank]
    for l in labels:
        out.append(l)
        out.append(blank)
    return np.array(out, dtype=int)


# ----------- CTC forward algorithm ----------------
def ctc_forward_log(probs, labels, blank=0):
    T, C = probs.shape
    ext = expand_with_blanks(labels, blank)
    S = len(ext)

    # convert to log domain
    logp = np.log(probs + 1e-12)

    # DP table
    alpha = np.full((T, S), -np.inf)

    # ---- Initialization ----
    alpha[0, 0] = logp[0, ext[0]]
    if S > 1:
        alpha[0, 1] = logp[0, ext[1]]

    # ---- Forward recursion ----
    for t in range(1, T):
        for s in range(S):
            candidates = [alpha[t-1, s]]  # stay

            if s > 0:
                candidates.append(alpha[t-1, s-1])  # move 1 step

            # skip 2 steps if allowed
            if s > 1 and ext[s] != blank and ext[s] != ext[s-2]:
                candidates.append(alpha[t-1, s-2])

            alpha[t, s] = logsumexp_vec(candidates) + logp[t, ext[s]]

    # ---- Final probability ----
    if S == 1:
        return alpha[T-1, 0]

    return logsumexp_vec([alpha[T-1, S-1], alpha[T-1, S-2]])


# ----------- example ----------
probs = np.array([
    [0.6, 0.4],   # t=0: blank=0.6, A=0.4
    [0.3, 0.7],   # t=1
    [0.2, 0.8]    # t=2
])

labels = [1]  # target = A (blank=0)

print("Log CTC probability:", ctc_forward_log(probs, labels))


#2. Without Log

In [ ]:
def expand_with_blanks(labels,blank=0):
  out=[blank]
  for l in labels:
    out.append(l)
    out.append(blank)
  return out

def ctc_forward(probs,labels,blank=0):
  T,C = probs.shape
  expanded = expand_with_blanks(labels,blank)
  S =len(expanded)
  alpha = np.zeros((T,S))

  #intialization of first frame
  # blank
  alpha[0,0] = probs[0,expanded[0]]
  if S>1: # if it is a character
    alpha[0,1] = probs[0,expanded[1]]


  for t in range(1,T):
    for s in range(S):
      #case 1: stay A-A
      candidate=[alpha[t-1,s]]

      #case 2: step A-blank OR Blank-A
      if s>0:
        candidate.append(alpha[t-1,s-1])

      #case 3: skip A-B --> skip blank
      if s>1 and expanded[s]!=blank and expanded[s]!=expanded[s-2]:
        candidate.append(alpha[t-1,s-2])

      alpha[t,s] = sum(candidate)*probs[t,expanded[s]]

      #case 4: length = 1
      if S==1:
        return alpha[T-1,0]
  return alpha[T-1,S-1]+alpha[T-1,S-2]

In [ ]:
probs = np.array([
    [0.6, 0.4],   # t=0: blank=0.6, A=0.4
    [0.3, 0.7],   # t=1
    [0.2, 0.8]    # t=2
])

labels = [1]  # target = A (blank=0)

print("CTC probability:", ctc_forward(probs, labels))

CTC probability: 0.868


#3. Greedy

In [ ]:
def ctc_greedy(probs,blank=3):
  T,C= probs.shape
  best_path=[]

  for t in range(T):
    max_index=0
    max_value=probs[t][0]
    for char in range(C):
      if probs[t][char]>max_value:
        max_value=probs[t][char]
        max_index =char


    best_path.append(max_index)
     # 2. remove repeats + blanks
    final_seq = []
    prev = None
    for c in best_path:
      if c!=blank and c!=prev:
        final_seq.append(c)
    return final_seq


In [ ]:
result = ctc_greedy(probs)
print("Decoded:", result)
print("Decoded labels:", [labels[i] for i in result])

Decoded: [0]
Decoded labels: ['a']


#4. BEAM 1: Using Dictionary--> Shorter

In [ ]:
def ctc_beam(probs,beam_size=2,blank=0):
  T ,C = probs.shape
  beam= {():1.0}   #{('a': prob) }
  for t in range(T):
    new_beams={}

    for seq ,score in beam.items():
        for char in range(C):
          p= probs[t,char]

          # Case 1:blank
          if char ==blank:
            new_seq = seq

          # Case 2: A->A
          elif len(seq)>0 and seq[-1] == char:
            new_seq = seq

          # Case 3: A->B
          else:
            new_seq= seq+(char,)
          new_prob = score*p

          # whether i will add 2 probs (eg, A-A AND A-BLANK) OR 1 Prob (A-B)
          if new_seq in new_beams:
           new_beams[new_seq]+=new_prob
          else:
           new_beams[new_seq] = new_prob

    beam = dict(sorted(new_beams.items(),key= lambda x:x[1],reverse = True)[:beam_size])
  return max(beam,key =beam.get)

In [ ]:
decoded = ctc_beam(probs, beam_size=2, blank=3)
print(decoded)


(0, 1, 2, 1)


#5. BEAM 2: Using List


In [ ]:
import numpy as np

def ctc_beam_search(probs, labels, beam_size=2, blank=3):
    T, C = probs.shape
    beams = [([], 1.0)]  # start: empty prefix with probability 1

    for t in range(T):
        new_beams = []

        for prefix, prob in beams:
            for c in range(C):
                p = probs[t][c]

                if c == blank:
                    new_prefix = prefix
                    new_prob = prob * p
                    new_beams.append((new_prefix, new_prob))
                else:
                    if len(prefix) > 0 and prefix[-1] == c:
                        new_prefix = prefix
                        new_prob = prob * p
                        new_beams.append((new_prefix, new_prob))
                    else:
                        new_prefix = prefix + [c]
                        new_prob = prob * p
                        new_beams.append((new_prefix, new_prob))

        # keep top beam_size sequences
        new_beams.sort(key=lambda x: x[1], reverse=True)
        beams = new_beams[:beam_size]

    best_prefix, best_prob = beams[0]
    return best_prefix


In [ ]:
labels = ["a", "b", "c", "blank"]

probs = np.array([
    [0.6, 0.2, 0.1, 0.1],  # t=0
    [0.1, 0.7, 0.1, 0.1],  # t=1
    [0.1, 0.1, 0.7, 0.1],  # t=2
    [0.1, 0.6, 0.1, 0.2],  # t=3
])

result = ctc_beam_search(probs, labels, beam_size=3)
print("Decoded:", result)


Decoded: [0, 1, 2, 1]


#6. Beam 3

In [ ]:
import numpy as np

def ctc_beam_search(probs, labels, beam_size=2, blank_idx=3):

    T, C = probs.shape
    beam = {(): 1.0}          # empty sequence with prob=1

    for t in range(T):
        new_beam = {}

        for seq, score in beam.items():
            for idx in range(C):
                p = probs[t, idx]
                ch = labels[idx]

                # Case 1: blank → same sequence
                if idx == blank_idx:
                    new_seq = seq

                # Case 2: repeat character (a→a)
                elif len(seq) > 0 and seq[-1] == ch:
                    new_seq = seq

                # Case 3: append new character
                else:
                    new_seq = seq + (ch,)

                new_prob = score * p

                # merge if sequence already exists
                if new_seq in new_beam:
                    new_beam[new_seq] += new_prob
                else:
                    new_beam[new_seq] = new_prob

        # keep top-K beams
        beam = dict(sorted(new_beam.items(), key=lambda x: x[1], reverse=True)[:beam_size])

        print(f"Time {t}:")
        print(beam)
        print()

    # return best final sequence
    return max(beam, key=beam.get)



In [ ]:

labels = ["a", "b", "c", "blank"]

log_probs = np.log(np.array([
    [0.6, 0.2, 0.1, 0.1],  # t = 0
    [0.1, 0.7, 0.1, 0.1],  # t = 1
    [0.1, 0.1, 0.7, 0.1],  # t = 2
    [0.1, 0.6, 0.1, 0.2],  # t = 3
]))
probs = np.exp(log_probs)

decoded = ctc_beam_search(probs, labels, beam_size=2, blank_idx=3)
print("Final Decoded Sequence:", decoded)

Time 0:
{('a',): np.float64(0.6), ('b',): np.float64(0.2)}

Time 1:
{('a', 'b'): np.float64(0.42), ('b',): np.float64(0.15999999999999998)}

Time 2:
{('a', 'b', 'c'): np.float64(0.294), ('b', 'c'): np.float64(0.11199999999999997)}

Time 3:
{('a', 'b', 'c', 'b'): np.float64(0.17639999999999997), ('a', 'b', 'c'): np.float64(0.0882)}

Final Decoded Sequence: ('a', 'b', 'c', 'b')
